In [306]:
from dotenv import load_dotenv
import os
load_dotenv("s3.env")
from hera.workflows import models, CronWorkflow, Workflow, script, Artifact, Parameter, DAG, Steps, Step, NoneArchiveStrategy
from hera.shared import global_config

global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "inca"
global_config.token = os.getenv("argo_token")
global_config.image = "ghcr.io/oscipal/image_zarr:latest"

security_context = {"runAsUser": 74268,
                    "runAsGroup": 71473}

nfs_volume = [models.Volume(
    name="eodc-mount",
    persistent_volume_claim={"claimName": "eodc-nfs-claim"},
    )]

In [ ]:
@script(volume_mounts=[models.VolumeMount(name="eodc-mount", mount_path="/eodc")])

def add_timestamps2zarr(store_path: str = "/eodc/private/openeo_platform/zarr_nacho/INCA_test.zarr"):
    import datetime
    import numpy as np
    import zarr

    now = datetime.datetime.now()
    now_np = np.datetime64(now).astype('datetime64[h]')
    origin = np.datetime64("2011-03-15T00:00:00").astype("datetime64[h]")

    new_shape = int((now_np-origin).astype(int))
    new_extent = np.arange(0,new_shape,1)

    store = zarr.storage.LocalStore(store_path)
    group = zarr.group(store=store)

    array_names=set(group.array_keys())
    coords = {"time", "x", "y"}
    data_arrays = array_names-coords

    group["time"].resize(new_shape)
    for array in data_arrays:
        group_shape  = group[array].shape
        group[array].resize((new_shape, group_shape[1], group_shape[2]))

    zarr.consolidate_metadata(store)
    store = zarr.storage.LocalStore(store_path)
    group = zarr.group(store=store)

    group["time"][:]=new_extent


In [298]:
@script(outputs=Artifact(name="inca-file", path="/tmp/INCA.nc", archive=NoneArchiveStrategy()))

def inca_download(variable: str):
    from urllib.request import urlretrieve
    import datetime

    ym = (datetime.date.today()-datetime.timedelta(days=8)).strftime("%Y%m")
    print(ym)
    url = f"https://public.hub.geosphere.at/datahub/resources/inca-v1-1h-1km/filelisting/{variable}/INCAL_HOURLY_{variable}_{ym}.nc"
    urlretrieve(url, f"/tmp/INCA.nc")

In [299]:
@script(inputs=Artifact(name="inca-file", path="/tmp/INCA.nc"),
        volume_mounts=[models.VolumeMount(name="eodc-mount", mount_path="/eodc")])

def inca_write(variable: str, store_path: str="/eodc/private/openeo_platform/zarr_nacho/INCA_test.zarr"):
    import xarray as xr
    import numpy as np
    import zarr

    artifact_path = f"/tmp/INCA.nc"

    def get_idx(array1, array2):
        min_idx = np.where(array1 == array2[0])[0][0]
        max_idx = np.where(array1 == array2[-1])[0][0] + 1
        return min_idx, max_idx

    data = xr.open_dataset(artifact_path, mask_and_scale=False).load()

    store = zarr.storage.LocalStore(store_path)
    group = zarr.group(store=store)
    x_extent = group["x"][:]
    y_extent = group["y"][:]

    x_min, x_max = get_idx(x_extent, data["x"].values)
    y_min, y_max = get_idx(y_extent, data["y"].values)

    origin = np.datetime64("2011-03-15T00:00:00").astype("datetime64[h]")
    time_min, time_max = data.time.values[0].astype("datetime64[h]"), data.time.values[-1].astype("datetime64[h]") + 1
    time_delta_min, time_delta_max = (time_min - origin).astype("int64"), (time_max - origin).astype("int64")

    group[variable][time_delta_min:time_delta_max, y_min:y_max, x_min:x_max] = data[variable].values

In [307]:
items = ["RR", "T2M", "TD2M", "P0", "UU", "VV", "RH2M", "GL"]

with CronWorkflow(
    generate_name="inca-zarr-",
    schedule="04 14 * * *",
    volumes = nfs_volume,
    security_context=security_context,
    entrypoint="workflow"
) as w:
    with DAG(name="pipeline", inputs=[Parameter(name="item")]) as pipeline:
        
        download = inca_download(arguments={"variable":"{{inputs.parameters.item}}"},
                                 )

        process = inca_write(arguments=[{"variable": "{{inputs.parameters.item}}"}, 
                                        download.get_artifact("inca-file").with_name("inca-file")],
                                        )

        download >> process

    with Steps(name="workflow"):
        add_timestamps2zarr()
        Step(name="parallel-pipelines", template=pipeline, with_param=items, arguments={"item":"{{item}}"})

In [ ]:
with open("INCA_workflow.yaml", "w") as f:
    f.write(w.to_yaml())

In [309]:
w.create()

CronWorkflow(api_version=None, kind=None, metadata=ObjectMeta(annotations=None, cluster_name=None, creation_timestamp=Time(__root__=datetime.datetime(2025, 7, 16, 14, 3, 57, tzinfo=datetime.timezone.utc)), deletion_grace_period_seconds=None, deletion_timestamp=None, finalizers=None, generate_name='inca-zarr-', generation=1, labels={'workflows.argoproj.io/creator': 'system-serviceaccount-default-jenkins'}, managed_fields=[ManagedFieldsEntry(api_version='argoproj.io/v1alpha1', fields_type='FieldsV1', fields_v1=FieldsV1(), manager='argo', operation='Update', subresource=None, time=Time(__root__=datetime.datetime(2025, 7, 16, 14, 3, 57, tzinfo=datetime.timezone.utc)))], name='inca-zarr-rfnfv', namespace='inca', owner_references=None, resource_version='310576193', self_link=None, uid='85494014-bb79-4958-a87b-b454633c567d'), spec=CronWorkflowSpec(concurrency_policy=None, failed_jobs_history_limit=None, schedule='04 14 * * *', starting_deadline_seconds=None, successful_jobs_history_limit=None